# Query CMIP6 ocean daily salinity data

# Notes

## imports

In [2]:
%%time
import glob
import json
import os
os.sys.path.insert(0, "/p/user_pub/xclim/persist/software/xsearch/")
import xcdat as xc
import xsearch as xs

import logging
logging.getLogger("xcdat").setLevel(logging.ERROR)

CPU times: user 5.99 s, sys: 1.71 s, total: 7.7 s
Wall time: 6.04 s


## use xsearch to find variables (in experiments) of interest - high frequency near-surface salinity¶

In [3]:
%%time
dpathsPi = xs.findPaths('piControl', 'so', 'mon', realm='ocean', cmipTable='Omon', deduplicate=True) # piControl monthly near-surface in the ocean realm
dpathsHi = xs.findPaths('historical', 'so', 'mon', realm='ocean', cmipTable='Omon', deduplicate=True) # historical monthly near-surface in the ocean realm
dpathsHiD = xs.findPaths('historical', 'sos', 'day', realm='ocean', cmipTable='Oday', deduplicate=True) # historical daily near-surface in the ocean realm

CPU times: user 68.8 ms, sys: 8.95 ms, total: 77.8 ms
Wall time: 182 ms


## loop over piControl entries and report top 10 levels

In [9]:
%%time
firstMemberOnly = True

models = xs.natural_sort(xs.getGroupValues(dpathsPi, 'model'))
for count, model in enumerate(models[0:1]):  # Toggle off the [0:1] single model layer to iterate over models
    dpaths_model = xs.retainDataByFacetValue(dpathsPi, 'model', model)
    members = xs.natural_sort(xs.getGroupValues(dpaths_model, 'member'))
    if firstMemberOnly:
        members = members[0:1]
    print(count, model, members)
    for member in members:
        dpaths_model_member_list = xs.getValuesForFacet(dpaths_model, 'member', member)
        if len(dpaths_model_member_list) > 1:
            print('Error: multiple paths detected for ', model, member, ': ', dpaths_model_member_list)
        else:
            dpath = dpaths_model_member_list[0]
            ncfile = xs.natural_sort(glob.glob(os.path.join(dpath, '*.nc')))[0]
        # now poll vertical axis
        ds = xc.open_dataset(ncfile, decode_times=False)
        ds
        #z = xc.get_dim_coords(ds, "Z")

0 ACCESS1-0 ['r1i1p1']
CPU times: user 194 ms, sys: 110 ms, total: 303 ms
Wall time: 7.87 s


In [10]:
ds

<xarray.Dataset>
Dimensions:       (time: 60, bnds: 2, lev: 50, j: 300, i: 360, vertices: 4)
Coordinates:
  * time          (time) float64 1.092e+05 1.093e+05 ... 1.11e+05 1.11e+05
  * lev           (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
  * j             (j) int32 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i             (i) int32 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    lat           (j, i) float32 -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    lon           (j, i) float32 80.5 81.5 82.5 83.5 ... 79.94 79.96 79.97 79.99
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds     (time, bnds) float64 ...
    lev_bnds      (lev, bnds) float64 ...
    lat_vertices  (j, i, vertices) float32 ...
    lon_vertices  (j, i, vertices) float32 ...
    so            (time, lev, j, i) float32 ...
    i_bnds        (i, bnds) float64 -0.5 0.5 0.5 1.5 ... 357.5 358.5 358.5 359.5
Attributes: (12/28)
    institution:            CSIRO (Commonwealth Scientific and Industrial Res...
    institute_id:           CSIRO-BOM
    experiment_id:          piControl
    source:                 ACCESS1-0 2011. Atmosphere: AGCM v1.0 (N96 grid-p...
    model_id:               ACCESS1-0
    forcing:                GHG, Oz, SA, Sl, Vl, BC, OC, (GHG = CO2, N2O, CH4...
    ...                     ...
    table_id:               Table Omon (27 April 2011) 694b38a3f68f18e58ba802...
    title:                  ACCESS1-0 model output prepared for CMIP5 pre-ind...
    parent_experiment:      N/A
    modeling_realm:         ocean
    realization:            1
    cmor_version:           2.8.0

In [11]:
ds.lev

<xarray.DataArray 'lev' (lev: 50)>
array([5.000000e+00, 1.500000e+01, 2.500000e+01, 3.500000e+01, 4.500000e+01,
       5.500000e+01, 6.500000e+01, 7.500000e+01, 8.500000e+01, 9.500000e+01,
       1.050000e+02, 1.150000e+02, 1.250000e+02, 1.350000e+02, 1.450000e+02,
       1.550000e+02, 1.650000e+02, 1.750000e+02, 1.850000e+02, 1.950000e+02,
       2.050000e+02, 2.168468e+02, 2.413490e+02, 2.807807e+02, 3.432505e+02,
       4.273156e+02, 5.367156e+02, 6.654141e+02, 8.127816e+02, 9.690651e+02,
       1.130935e+03, 1.289605e+03, 1.455770e+03, 1.622926e+03, 1.801558e+03,
       1.984855e+03, 2.182905e+03, 2.388417e+03, 2.610935e+03, 2.842564e+03,
       3.092205e+03, 3.351295e+03, 3.628058e+03, 3.913264e+03, 4.214495e+03,
       4.521918e+03, 4.842566e+03, 5.166130e+03, 5.499245e+03, 5.831294e+03])
Coordinates:
  * lev      (lev) float64 5.0 15.0 25.0 35.0 ... 5.166e+03 5.499e+03 5.831e+03
Attributes:
    bounds:         lev_bnds
    units:          m
    axis:           Z
    positive:       down
    long_name:      ocean depth coordinate
    standard_name:  depth